In [1]:
import os

os.environ['R_HOME'] = '/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/R'

In [2]:
from SPACEL.setting import set_environ_seed
set_environ_seed()
from SPACEL import Spoint
import scanpy as sc
import numpy as np
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.serif'] = ['Arial']
sc.settings.set_figure_params(dpi=50,dpi_save=300,facecolor='white',fontsize=10,vector_friendly=True,figsize=(3,3))
sc.settings.verbosity = 3

Setting environment seed: 42


/home/comp/cszrwang/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


In [3]:
sc_ad = sc.read_h5ad('/home/comp/cszrwang/data/Breast/spadecon/gm_input_sc.h5ad')
st_ad = sc.read_h5ad('/home/comp/cszrwang/data/Breast/spadecon/gm_input_st.h5ad')

sc.pp.filter_genes(st_ad,min_cells=1)
sc.pp.filter_genes(sc_ad,min_cells=1)
sc.pp.filter_cells(st_ad,min_genes=1)
sc.pp.filter_cells(sc_ad,min_genes=1)

In [4]:
sc_ad.obs['celltype']

0              Endothelial
1              Endothelial
2              Endothelial
3              Endothelial
4              Endothelial
               ...        
11482    Normal Epithelial
11483    Normal Epithelial
11484    Normal Epithelial
11485    Normal Epithelial
11486    Normal Epithelial
Name: celltype, Length: 11487, dtype: category
Categories (8, object): ['B-cells', 'CAFs', 'Cancer Epithelial', 'Endothelial', 'Myeloid', 'Normal Epithelial', 'PVL', 'T-cells']

In [5]:
spoint_model = Spoint.init_model(sc_ad,st_ad,celltype_key='celltype',deg_method='t-test',sm_size=100000,use_gpu=True)

Setting global seed: 42
normalizing counts per cell
    finished (0:00:01)
normalizing counts per cell
    finished (0:00:00)
### Finding marker genes...
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:13)
celltype
B-cells              200
CAFs                 200
Cancer Epithelial    200
Endothelial          200
Myeloid              200
Normal Epithelial    200
PVL                  200
T-cells              200
Name: count, dtype: int64
### Used gene numbers: 1532
### Initializing sample probability
### Genetating simulated spatial data using scRNA data with mode: unbalance
### Genetating simulated spatial data using scRNA data with mode: sqrt
### Genetating s

In [6]:
spoint_model.train(max_steps=5000, batch_size=1024)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Epoch 100/100: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:41<00:00,  1.05s/it, loss=1.1e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 100/100: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:41<00:00,  1.01s/it, loss=1.1e+03, v_num=1]


Step 5000: Test inference loss=-0.957: 100%|████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [28:02<00:00,  2.97it/s]


In [7]:
pre = spoint_model.deconv_spatial()

In [8]:
pre

,T-cells,CAFs,Normal Epithelial,Endothelial,PVL,Cancer Epithelial,Myeloid,B-cells
AAACAAGTATCTCCCA-1,0.233064,0.282882,0.099184,0.062582,0.000000,0.000000,0.185919,0.128334
AAACACCAATAACTGC-1,0.053455,0.030229,0.000000,0.214020,0.134593,0.392615,0.164994,0.000000
AAACAGAGCGACTCCT-1,0.078407,0.208894,0.081844,0.128832,0.117009,0.081047,0.106765,0.197203
AAACAGGGTCTATATT-1,0.242454,0.147347,0.013032,0.090921,0.000000,0.000000,0.179752,0.325613
AAACAGTGTTCCTGGG-1,0.000000,0.309359,0.000000,0.210582,0.000000,0.312490,0.157917,0.000000
...,...,...,...,...,...,...,...,...
TTGTTGTGTGTCAAGA-1,0.059999,0.333702,0.000000,0.015403,0.000000,0.362640,0.209971,0.000000
TTGTTTCACATCCAGG-1,0.053427,0.233546,0.000000,0.024102,0.108724,0.231202,0.340189,0.000000
TTGTTTCATTAGTCTA-1,0.013918,0.176223,0.000000,0.243918,0.098602,0.314399,0.149185,0.000000
TTGTTTCCATACAACT-1,0.283598,0.303393,0.000000,0.123174,0.041805,0.116785,0.124410,0.000000


In [12]:
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/Breast_1/SPACEL/"

In [13]:
pre.to_csv(output_file_path + '/SPACEL_result.csv')